### Load SNOTEL Data from the sites around Juneau

Notebook contents 
* Heen Latinee, AK (#1270)
* Long Lake, AK (#1001) 
* Honah, AK (#1318)

created by Cassie Lumbrazo\
last updated: Oct 2025\
run location: UAS linux\
python environment: **xarray**

In [2]:
# import packages 
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

sns.set_theme()
# plt.rcParams['figure.figsize'] = [12,6] #overriding size

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import scipy

In [3]:
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from matplotlib import ticker

In [3]:
pwd

'/home/cassie/python/repos/juneau_data_analysis'